## Start of project. Finding all the urls to each city on their website

In [1]:
import requests

URL = "https://www.lennar.com/"
page = requests.get(URL)

# print(page.text)

In [2]:
from bs4 import BeautifulSoup

In [3]:
soup = BeautifulSoup(page.content, "html.parser")

In [4]:
# page.text

In [5]:
results = soup.find(id="__NEXT_DATA__")

In [6]:
import json

In [7]:
data = json.loads(results.text)

In [8]:
markets = dict()
states = dict()

In [9]:
for element in data['props']['pageProps']['initialApolloState']:
    if element.startswith("Market"):
        tmp = data['props']['pageProps']['initialApolloState'][element]
        # print(tmp['id'])
        # print(tmp['name'])
        # print(tmp['slug'])
        
#        markets[tmp['slug']] = tmp['id']
        markets.setdefault(tmp['slug'], []).append(tmp['id'])

        
    elif element.startswith("State"):
        tmp = data['props']['pageProps']['initialApolloState'][element]
        # print(tmp['id'])
        # print([x.get('__ref').split("MarketType:")[1] for x in tmp['markets']])
        
        states[tmp['slug']] = [x.get('__ref').split("MarketType:")[1] for x in tmp['markets']]
        
    else:
        continue

In [10]:
len(states)

26

In [11]:
len(markets)

72

In [12]:
# found repeats
# brought to same site under lenar even though they're listed under different states
# we'll reflect that in the state list

In [13]:
{i for i in markets if len(markets[i])==2}

{'charlotte', 'greater-philadelphia-area', 'inland-northwest'}

In [14]:
states

{'alabama': ['mk_161', 'mk_160'],
 'arizona': ['mk_54', 'mk_74'],
 'california': ['mk_148',
  'mk_32',
  'mk_37',
  'mk_18',
  'mk_63',
  'mk_66',
  'mk_67'],
 'colorado': ['mk_17'],
 'delaware': ['mk_112', 'mk_131'],
 'florida': ['mk_24',
  'mk_34',
  'mk_40',
  'mk_25',
  'mk_50',
  'mk_52',
  'mk_55',
  'mk_109',
  'mk_134',
  'mk_65',
  'mk_71',
  'mk_72',
  'mk_103'],
 'illinois': ['mk_9'],
 'maryland': ['mk_145', 'mk_144', 'mk_70', 'mk_146'],
 'minnesota': ['mk_42'],
 'north-carolina': ['mk_7', 'mk_154', 'mk_57', 'mk_141'],
 'new-jersey': ['mk_147'],
 'nevada': ['mk_38', 'mk_58'],
 'pennsylvania': ['mk_140', 'mk_100', 'mk_80'],
 'south-carolina': ['mk_10', 'mk_8', 'mk_159', 'mk_26', 'mk_152', 'mk_47'],
 'texas': ['mk_1', 'mk_163', 'mk_14', 'mk_28', 'mk_64'],
 'virginia': ['mk_157', 'mk_49', 'mk_77'],
 'georgia': ['mk_82', 'mk_151'],
 'washington': ['mk_156', 'mk_86', 'mk_87'],
 'oregon': ['mk_165', 'mk_88', 'mk_158'],
 'tennessee': ['mk_93'],
 'indiana': ['mk_117', 'mk_115'],
 'u

In [15]:
data = dict()
for name, lst in markets.items():
    data[name] = [s for s in states if lst[0] in states[s]][0]

In [16]:
data

{'coastal-alabama': 'alabama',
 'huntsville': 'alabama',
 'phoenix': 'arizona',
 'tucson': 'arizona',
 'central-california': 'california',
 'inland-empire': 'california',
 'la-orange-county': 'california',
 'palm-springs-coachella': 'california',
 'sacramento': 'california',
 'san-diego': 'california',
 'san-francisco-bay-area': 'california',
 'denver': 'colorado',
 'greater-philadelphia-area': 'delaware',
 'sussex-county': 'delaware',
 'ft-lauderdale': 'florida',
 'jacksonville-st-augustine': 'florida',
 'miami': 'florida',
 'naples-ft-myers': 'florida',
 'ocala': 'florida',
 'orlando': 'florida',
 'palm-beach': 'florida',
 'pensacola': 'florida',
 'perdido-key': 'florida',
 'sarasota-manatee': 'florida',
 'space-coast-melbourne': 'florida',
 'tampa-manatee': 'florida',
 'treasure-coast': 'florida',
 'chicago': 'illinois',
 'central-maryland': 'maryland',
 'eastern-shore': 'maryland',
 'md-dc-metro': 'maryland',
 'southern-maryland': 'maryland',
 'minneapolis-st-paul': 'minnesota',
 '

In [17]:
import pandas as pd

In [18]:
df = pd.DataFrame(list(data.items()), columns = ["market", "state"])

In [19]:
df['url'] = "https://www.lennar.com/new-homes/"+df["state"]+"/"+df["market"]

In [20]:
df.head()

,market,state,url
0,coastal-alabama,alabama,https://www.lennar.com/new-homes/alabama/coast...
1,huntsville,alabama,https://www.lennar.com/new-homes/alabama/hunts...
2,phoenix,arizona,https://www.lennar.com/new-homes/arizona/phoenix
3,tucson,arizona,https://www.lennar.com/new-homes/arizona/tucson
4,central-california,california,https://www.lennar.com/new-homes/california/ce...


In [21]:
df.url[0]

'https://www.lennar.com/new-homes/alabama/coastal-alabama'

In [23]:
df.to_csv('./tmp.csv', index=False)